# Колодий Антон, гр. 8376

Поскольку мне не очень понравился датасет, который я выбрал для второго задания, то для третьей работы я выбрал новый. 
Он содержит информацию о звездах. Дополнительно я проверил две звезды, не входящие в датасет.

Задача, которая ставится перед моделью классификации: предсказать, к какому типу звёзд относится некоторая звезда.

In [233]:
import numpy as np
import pandas as pd

In [234]:
df = pd.read_csv('stars.csv')
df.head()

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0


Целевой класс - Type. Это тип звезды от 0 до 5

* Красный карлик - 0
* Коричневый карлик - 1
* Белый карлик - 2
* Основная последовательность - 3
* Супергиганты - 4
* Гипергиганты - 5

# Задание
Необходимо оценить и сравнить результаты классификации, используя следующие
алгоритмы классификации:
* kNN
* дерево решений

Стандартизируем набор данных

In [235]:
from sklearn.preprocessing import StandardScaler

In [236]:
x = df.drop("Type", axis=1)
#Создадим фиктивные переменные, поскольку у нас 2 атрибута - цвет, спектральный класс - текстовые
x=pd.get_dummies(x,drop_first=True)

#Стандартизация набора данных
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)

Разделим датасет на обучающую и тестовую выборки

In [237]:
from sklearn.model_selection import train_test_split

In [238]:
y = df['Type']
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.2)

# Обучение модели K-ближайших соседей

In [239]:
from sklearn.neighbors import KNeighborsClassifier

In [240]:
model = KNeighborsClassifier(n_neighbors = 3)
model.fit(x_training_data, y_training_data)
predictions = model.predict(x_test_data)
predictions_prob = model.predict_proba(x_test_data)

Результаты классификации:

In [241]:
from sklearn.metrics import classification_report

In [242]:
print(classification_report(y_test_data, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00        10
           2       0.89      1.00      0.94         8
           3       1.00      1.00      1.00        11
           4       1.00      0.86      0.92         7
           5       1.00      1.00      1.00         7

    accuracy                           0.98        48
   macro avg       0.98      0.98      0.98        48
weighted avg       0.98      0.98      0.98        48



ROC:

In [243]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [244]:
roc_auc = roc_auc_score(y_test_data, predictions_prob, multi_class='ovr')
print('ROC AUC=%.3f' % (roc_auc))

ROC AUC=1.000


# Обучение модели Дерево решений

In [245]:
from sklearn.tree import DecisionTreeClassifier

In [246]:
model_tree = DecisionTreeClassifier()
model_tree.fit(x_training_data, y_training_data)
predictions_tree = model_tree.predict(x_test_data)
predictions_tree_prob = model_tree.predict_proba(x_test_data)

In [247]:
print(classification_report(y_test_data, predictions_tree))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00        11
           4       1.00      1.00      1.00         7
           5       1.00      1.00      1.00         7

    accuracy                           1.00        48
   macro avg       1.00      1.00      1.00        48
weighted avg       1.00      1.00      1.00        48



ROC:

In [248]:
roc_auc_tree = roc_auc_score(y_test_data, predictions_tree_prob, multi_class='ovr')
print('ROC AUC=%.3f' % (roc_auc))

ROC AUC=1.000


# Дополнительно от меня

Проверим наши модели. Возьмем данные по этой звезде: https://en.wikipedia.org/wiki/HD_202628
* Температура = 5843
* L = 0.951
* R = 0.951
* A_M = 4.856
* color = yellowish
* Spectral_Class = G
Мы как бы не знаем, что звезда относится к классу главной последовательности (3) и хотим, чтобы модель предсказала нам это.

In [249]:
star = [[5843, 0.951, 0.951, 4.856, 'yellowish','G']]

In [255]:
def star_type(star, model):
    #Добавим нашу звезду к датафрейму для удобства извлечения после добавления фиктивных переменных
    header = df.drop("Type", axis=1).columns.values
    t_df = pd.DataFrame(star, columns=header)
    t_df = df.drop("Type", axis=1).append(t_df)
    
    #Создадим фиктивные переменные, поскольку у нас 2 атрибута - цвет, спектральный класс - текстовые
    value=pd.get_dummies(t_df,drop_first=True)
    
    #Стандартизация набора данных
    scaler = StandardScaler()
    scaler.fit(value)
    value = scaler.transform(value)
    value = value[-1]
    _type = model.predict(value.reshape(1,-1))
    
    if (_type == 0):
        print("Красный карлик",_type)
    elif (_type == 1):
        print("Коричневый карлик",_type)
    elif (_type == 2):
        print("Белый карлик",_type)
    elif (_type == 3):
        print("Звезда главной последовательности",_type)
    elif (_type == 4):
        print("Супер-гигант",_type)
    else:
        print("Гипер-гигант",_type)
        
    

In [256]:
print("Метод Дерево решений:")
star_type(star, model_tree)
print()
print("Метод K-ближайших соседей:")
star_type(star, model)

Метод Дерево решений:
Гипер-гигант [5]

Метод K-ближайших соседей:
Белый карлик [2]


Хочу проверить еще одну звезду, но теперь методом K-ближайших соседей: https/en.wikipedia.org/wiki/HD_168625
* Температура = 14000
* L = 380000
* R = 105
* A_M = -8.39
* color = Blue
* Spectral_Class = B


In [257]:
star = [[14000, 380000, 105, -8.39, 'Blue','B']]
print("Метод Дерево решений:")
star_type(star, model_tree)
print()
print("Метод K-ближайших соседей:")
star_type(star, model)

Метод Дерево решений:
Гипер-гигант [5]

Метод K-ближайших соседей:
Белый карлик [2]


# Итог по дополнительному
Дерево решений точно определило оба типа звезд: первая была звезда главной последвательности, вторая - Гипер-гигант.
Метод k-ближайших соседей неверно определил вторую звезду, отнёс её к классу белый карлик.


# Выводы по работе
Accuracy, precision, recall и f1 - единица у дерева.

У модели k-ближайших соедей для разных типов звезд precision, recall и f1 находятся в диапазоне от 0.86 до 1.
Для этой модели необходимо точнее подбирать параметр соседей, я выбирал 1, а потом 3. Ни один не показывает идеальную точность (0.96 и 0.98 соответственно) Последний вариант параметра (3) представлен в данный момент.

Площадь под кривой ROC для обоих моделей оказалась 1. Но здесь либо порешность вычисления, либо я ошибся в написании кода, поскольку модель k-ближайших соседей ложно срабатывает на примере Гипер-гиганта...
